In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
os.listdir('../input/aerial-cactus-identification/')

In [ ]:
os.listdir('../input/aerial-cactus-identification/train/train')

In [ ]:
train_dir = '../input/aerial-cactus-identification/train/train'

In [ ]:
csv_path = '../input/aerial-cactus-identification/train.csv'

df = pd.read_csv(csv_path)

In [ ]:
df.head()

In [ ]:
filenames = df['id']
filenames.head()

In [ ]:
file_paths = [os.path.join(train_dir, fname) for fname in filenames]
file_paths[:5]

In [ ]:
train_df = pd.DataFrame(data={'id': file_paths, 'has_cactus': df['has_cactus']})
train_df.head()

In [ ]:
train_df = train_df.astype(np.str)

In [ ]:
sample_csv_path = '../input/aerial-cactus-identification/sample_submission.csv'
sample_df = pd.read_csv(sample_csv_path)
sample_df.head()

In [ ]:
len(train_df)

In [ ]:
train_df = train_df[:-500]
test_df = train_df[-500:]

len(train_df), len(test_df)

In [ ]:
path = train_df['id'][0]

# Load Packages

In [ ]:
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir

In [ ]:
test_dir = '../input/aerial-cactus-identification/test/test/'

In [ ]:
os.listdir(test_dir)

# Data Explore

In [ ]:
train_df.head()

In [ ]:
img_pil = Image.open(path)
image = np.array(img_pil)

image.shape

In [ ]:
plt.imshow(image)
plt.show()

# Hyperparameter

In [ ]:
input_shape = (32, 32, 3)
batch_size = 32
num_classes = 2
num_epochs = 10

learning_rate = 0.01

# Model

In [ ]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(64, (3, 3), padding='same')(inputs)
net = layers.Conv2D(64, (3, 3), padding='same')(net)
net = layers.Conv2D(64, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)

net = layers.Conv2D(128, (3, 3), padding='same')(net)
net = layers.Conv2D(128, (3, 3), padding='same')(net)
net = layers.Conv2D(128, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(256, (3, 3), padding='same')(net)
net = layers.Conv2D(256, (3, 3), padding='same')(net)
net = layers.Conv2D(256, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.Conv2D(512, (3, 3), padding='same')(net)
net = layers.BatchNormalization()(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['accuracy'])

# Data Preprocess

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='id',
    y_col='has_cactus',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='id',
    y_col='has_cactus',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='sparse'
)

# Train

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

# Evaluate

In [ ]:
test_dir

In [ ]:
sample_df.head()

In [ ]:
path = os.path.join(test_dir, sample_df['id'][0])

In [ ]:
img_pil = Image.open(path)
image = np.array(img_pil)
image.shape

In [ ]:
plt.imshow(image)
plt.show()

In [ ]:
pred = model.predict(image[tf.newaxis, ...])
pred

In [ ]:
pred = np.argmax(pred)
pred

In [ ]:
preds = []

for fname in tqdm_notebook(sample_df['id']):
    path = os.path.join(test_dir, fname)

    img_pil = Image.open(path)
    image = np.array(img_pil)

    pred = model.predict(image[tf.newaxis, ...])
    pred = np.argmax(pred)
    preds.append(pred)


In [ ]:
submission_df = pd.DataFrame(data={'id': sample_df['id'], 'has_cactus': preds})

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('samplesubmission.csv', index=False)

In [ ]:
os.listdir()